In [ ]:
import numpy as np
from numpy.fft import fft2, ifft2
import matplotlib.pyplot as plt
from itertools import permutations
from astropy.io import fits
import os as os
import operator
%matplotlib nbagg


def colourbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    ax = mappable.axes
    figle = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return figle.colorbar(mappable, cax=cax, format='%g')


# TODO: start maths


length = 200

root = '/home/broughtonco/documents/nrc/data/IC348/'
files = os.listdir(root)

FirstEpoch = fits.open('/home/broughtonco/documents/nrc/data/IC348/IC348_20151222_00019_850_EA3_cal.fit')
FirstEpochDate = FirstEpoch[0].header['UTDATE']
FirstEpochData = FirstEpoch[0].data[0]
FED_MidMapX, FED_MidMapY = FirstEpochData.shape[1] // 2 - 1, FirstEpochData.shape[0] // 2 - 1
FirstEpochData = FirstEpochData[FED_MidMapY-length:FED_MidMapY+length, FED_MidMapX-length:FED_MidMapX+length]

DataSetsPSD, DataSetsAC, radiuses = [], [], []
Dat_Titles = ['Map', 'PowerSpectrum', 'AutoCorr', 'XCorr']
dictionary1 = {}
# DataSets
for fn in files:
    if os.path.isfile(root+'/'+fn) and fn[-4:] != '.txt':
        hdul = fits.open(root+'/'+fn)
        date = hdul[0].header['UTDATE']
        obj = hdul[0].header['OBJECT']
        dictionary1[date] = hdul[0]

for item in sorted(dictionary1.items(), key=operator.itemgetter(0)):
    G2D = item[1].data[0]
    date = item[1].header['UTDATE']
    obj = item[1].header['OBJECT']
    MidMapX, MidMapY = G2D.shape[1] // 2 - 1, G2D.shape[0] // 2 - 1

    Clipped_G2D = G2D[MidMapY-length:MidMapY+length, MidMapX-length:MidMapX+length]
    ClippedMidMapX, ClippedMidMapY = Clipped_G2D.shape[1] // 2 - 1, Clipped_G2D.shape[0] // 2 - 1

    XCorr = ifft2(fft2(Clipped_G2D) * fft2(FirstEpochData).conj())
    XCorr_rolled = np.roll(XCorr, XCorr.shape[0]//2-1, axis=(0, 1))

    PSD = fft2(Clipped_G2D) * fft2(Clipped_G2D).conj()
    PSD_rolled = np.roll(PSD, PSD.shape[0]//2-1, axis=(0, 1))

    AC = ifft2(PSD)
    AC_rolled = np.roll(AC.real, AC.shape[0]//2-1, axis=(0, 1))

    loc = list(permutations(np.arange(0, Clipped_G2D.shape[0]), 2))

    radius, PSD_pows, AC_pows = [0], [PSD_rolled[0, 0].real], [AC_rolled[0, 0].real]
    # writing to a fits file
    dat = [Clipped_G2D.real, PSD_rolled.real, AC_rolled.real, XCorr_rolled.real]

    for data, name in zip(dat, Dat_Titles):
        if os.path.exists('/home/broughtonco/documents/nrc/data/IC348/GeneratedMapsFits/{}_{}_{}.fit'.format(
                obj, date, name)):
            os.remove('/home/broughtonco/documents/nrc/data/IC348/GeneratedMapsFits/{}_{}_{}.fit'.format(
                obj, date, name))
        hdu = fits.PrimaryHDU(data)
        hdu.writeto(
            '/home/broughtonco/documents/nrc/data/IC348/GeneratedMapsFits/{}_{}_{}.fit'.format(obj, date, name))

    for idx in loc:
        r = ((idx[0] - ClippedMidMapX) ** 2 + (idx[1] - ClippedMidMapY) ** 2) ** (1 / 2)
        PSD_pow = PSD_rolled[idx[0], idx[1]].real
        AC_pow = AC_rolled[idx[0], idx[1]].real
        radius.append(r)
        PSD_pows.append(PSD_pow)
        AC_pows.append(AC_pow)

    DataSetsAC.append(np.array(AC_pows))
    DataSetsPSD.append(np.array(PSD_pows))

    AC_rolled[AC_rolled.shape[0]//2-1, AC_rolled.shape[1]//2-1] = 0

    fig1 = plt.figure(figsize=(20, 20))
    grid = plt.GridSpec(5, 4, hspace=0.4, wspace=0.2)

    # row 1

    MapIM = fig1.add_subplot(grid[0, 0])
    acIM = fig1.add_subplot(grid[0, 1])
    psdIM = fig1.add_subplot(grid[0, 2])
    xcorrIM = fig1.add_subplot(grid[0, 3])

    # row 2 + 3

    AvgScatterPSD = fig1.add_subplot(grid[1:3, :])

    # row 3 + 4

    AvgScatterAC = fig1.add_subplot(grid[3:5, :])

    acIM.get_shared_x_axes().join(acIM, psdIM)
    acIM.get_shared_y_axes().join(acIM, psdIM)

    # row 1

    MapIM.set_title('Map')
    im1 = MapIM.imshow(Clipped_G2D, origin='lower', cmap='magma')
    colourbar(im1)

    acIM.set_title('Auto-correlation')
    im2 = acIM.imshow(AC_rolled.real, origin='lower', cmap='magma')
    colourbar(im2)

    psdIM.set_title('Power Spectrum')
    im3 = psdIM.imshow(PSD_rolled.real, origin='lower', cmap='magma')
    colourbar(im3)

    xcorrIM.set_title('X-Correlation')
    im4 = xcorrIM.imshow(XCorr_rolled.real, origin='lower', cmap='magma')
    colourbar(im4)

    # row 3

    AvgScatterPSD.set_title('Dispersion of Signal power at a a given frequency')
    AvgScatterPSD.set_xlabel('Frequency')
    AvgScatterPSD.set_ylabel('Power')
    AvgScatterPSD.set_yscale('log')
    AvgScatterPSD.set_ylim(bottom=10**-7, top=10**7)
    im5 = AvgScatterPSD.scatter(radius, PSD_pows, marker=',', lw=0, alpha=0.3, color='red', s=1)

    AvgScatterAC.set_title('Auto Correlation')
    AvgScatterAC.set_xlabel('place holder')
    AvgScatterAC.set_ylabel('p-h')
    AvgScatterAC.set_yscale('log')
    AvgScatterAC.set_aspect('auto')
    AvgScatterAC.set_ylim(bottom=10**-4, top=10**4)
    im6 = AvgScatterAC.scatter(radius, AC_pows, marker=',', lw=0, alpha=0.3, color='red', s=1)

    fig1.suptitle('{} IC348 epoch'.format(date))

    # plt.show()
    fig1.savefig('/home/broughtonco/documents/nrc/data/IC348/IC348_plots/{}'.format(date))

np.save('/home/broughtonco/documents/nrc/data/IC348/Datafiles/IC348_PSD.npy', np.array(DataSetsPSD))
np.save('/home/broughtonco/documents/nrc/data/IC348/Datafiles/IC348_AC.npy', np.array(DataSetsAC))
# noinspection PyUnboundLocalVariable
np.save('/home/broughtonco/documents/nrc/data/IC348/Datafiles/IC348_radius.npy', np.array(radius))
